In [1]:
from src.core import *
from src import DSL
import inspect
import typing

import asyncio

from poke_env.player.random_player import RandomPlayer
from poke_env.player.utils import cross_evaluate
from poke_env.player_configuration import PlayerConfiguration
from poke_env.server_configuration import LocalhostServerConfiguration
from poke_env.utils import to_id_str
from tabulate import tabulate

import trueskill
import copy
from trueskill import Rating, rate_1vs1, quality_1vs1
from tqdm.notebook import tqdm
import itertools
import numpy as np
import pandas as pd
from pprint import pprint


import logging
# from aiologger.handlers.files import AsyncFileHandler
# from aiologger.handlers.files import AsyncFileHandler
from tempfile import NamedTemporaryFile

from pathlib import Path
import pickle
import dill

dsl.gets_stab return type hint not valid.
[None, ['dsl.check_move_is_gyro_ball', 'move'], ['dsl.check_move_rainy', 'move'], ['dsl.check_move_sds_always', 'move'], ['dsl.check_move_sds_if_hits_opp', 'move'], ['dsl.check_move_sunny', 'move'], None, ['dsl.move_base_power', 'move', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.move_is_physical', 'move'], ['dsl.move_is_special', 'move'], ['dsl.move_is_status', 'move'], ['dsl.move_type', 'move', <RULE.ENUM_COMPARATOR: 17>, <RULE.POKEMON_TYPE: 18>], ['dsl.opp_base_defense', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.opp_base_spec_defense', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.opp_speed', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.player_base_attack', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.player_base_spec_defense', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.player_base_speed', <RULE.NUM_COMPARATOR: 16>, <RULE.INT_NUM: 15>], ['dsl.type_multiplier', 'move', <RULE.NUM_COMPA

In [64]:
class PlayerWrapper(Player):
    def __init__(self, *args, script=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.script = script
        
#         self.log_file = NamedTemporaryFile()
#         self.patch_logger()
    
#     def patch_logger(self):
#         self.logger.removeHandler(self.logger.handlers[0])
#         self.logger.addHandler(logging.FileHandler(self.log_file.name))
#         self.logger.
        
    def choose_move(self, battle):
        try:
            move = self.script.choose_move(battle)
            return self.create_order(move)
        except Exception as e:
            print(e)
            return self.choose_random_move(battle)
            
    @classmethod
    def from_script(cls, script):
#         script_name = script.__class__.__name__.replace('_', '')
        script_name = get_node_id()
#         user_name = 
        return cls(
            script=script,
            player_configuration=PlayerConfiguration(script_name, None),
            battle_format="gen7randombattle",
            server_configuration=LocalhostServerConfiguration,
            max_concurrent_battles=100000000,
#             log_level=logging.DEBUG,
        )

In [3]:
async def evaluate_population(population, num_games, verbose=True):
    players = population[:]
    players_lookup = {player.username: player for player in players}
    
    for _ in tqdm(range(num_games), disable=not verbose):
        random.shuffle(players)
        awaitables = []
        for p1, p2 in zip(players[0::2], players[1::2]):
            send = p1.send_challenges(
                opponent=to_id_str(p2.username),
                n_challenges=1,
                to_wait=p2.logged_in,
            )
            accept = p2.accept_challenges(
                opponent=to_id_str(p1.username),
                n_challenges=1,
            )
            awaitables.append(send)
            awaitables.append(accept)
        await asyncio.gather(*awaitables)
        
#     battles = sum((list(player.battles.values()) for player in players), [])
    
    # use TrueSkill
#     for battle in battles:
#         p1_dict, p2_dict = battle.players
#         p1 = players_lookup[p1_dict['username']]
#         p2 = players_lookup[p2_dict['username']]
        
#         if battle.won:
#             winner, loser = p1, p2
#         else:
#             winner, loser = p2, p1
#         winner.rating, loser.rating = rate_1vs1(winner.rating, loser.rating)
    

In [4]:
def sort_population(population):
    return sorted(population, key=lambda p: p.rating.mu, reverse=True)

# def sort_population(population):
#     return sorted(population, key=lambda p: p.n_won_battles, reverse=True)

In [5]:
def get_elites(population, num_elites):
    return sort_population(population)[:num_elites]

In [6]:
def get_tournament_elites(population, tournament_size, num_elites):
    tournament = random.sample(population, tournament_size)
    return get_elites(tournament, num_elites)

In [7]:
def mutate(tree, filter_=None):
    tree = copy.deepcopy(tree)
    candidates = anytree.search.findall(tree, filter_=filter_)
    if candidates:
        target = random.choice(candidates)
        target.children = []
        generate_tree(target)
    return tree

In [8]:
def crossover(left, right):
    def _is_valid_subtree_head(node):
#         return node.name in [RULE.IF_BLOCK]
        return True

    def _find_head_types(root):
        return [node.name for node in anytree.search.findall(root, filter_=_is_valid_subtree_head)]

    def _select_type_node(root, type_):
        candidates = anytree.search.findall(root, filter_=lambda node: node.name == type_)
        return random.choice(candidates)
    
    def _swap_children(left, right):
        left.children, right.children = right.children, left.children
        
    left = copy.deepcopy(left)
    right = copy.deepcopy(right)

    left_head_types = _find_head_types(left)
    right_head_types = _find_head_types(right)

    type_intersection = list(set(left_head_types) & set(right_head_types))
    if type_intersection:
        type_to_swap = random.choice(type_intersection)
        left_head = _select_type_node(left, type_to_swap)
        right_head = _select_type_node(right, type_to_swap)
        _swap_children(left_head, right_head)
    return left, right

In [9]:
def get_random_population(k):
    return [PlayerWrapper.from_script(exec_tree(get_random_tree()))
            for _ in range(k)]

In [10]:
def get_population_stats(population):
    ratings = pd.Series([player.rating.mu for player in population])
    return ratings.describe()

In [11]:
def assign_trueskill(population):
    for player in population:
        player.rating = Rating()

    player_lookup = {player.username: player for player in population}
    battles = sum((list(player.battles.values()) for player in population), [])

    for battle in battles:
        player = player_lookup[battle.player_username]
        oppo = player_lookup[battle._opponent_username]

        if battle.won:
            winner, loser = player, oppo
        else:
            winner, loser = oppo, player
        winner.rating, loser.rating = rate_1vs1(winner.rating, loser.rating)

In [20]:
def save_population(population, fname):
    path = Path('temp_scripts').joinpath(fname)
    with open(path, 'wb') as f:
        dill.dump([player.script.tree for player in population], f)
            
def load_population(fname):
    path = Path('temp_scripts').joinpath(fname)
    with open(path, 'rb') as f:
        return [PlayerWrapper.from_script(exec_tree(tree)) for tree in dill.load(f)]

# Evolve

In [60]:
# large
num_elites = 12
tournament_size = 8
population_cap = 128
epochs = 50
num_games = 20

In [66]:
# small
num_elites = 2
tournament_size = 4
population_cap = 64
epochs = 50
num_games = 10

In [67]:
population = get_random_population(k=population_cap)
generations = []
for i in tqdm(range(epochs)):
    save_population(population, f"{i}.dill")
    await evaluate_population(population, num_games=num_games, verbose=False)
    assign_trueskill(population)
    generations.append([copy.deepcopy(p.script) for p in population])
    population = sort_population(population)
    next_population = []
    for elite in get_elites(population, num_elites):
        new_player = PlayerWrapper.from_script(elite.script)
        next_population.append(new_player)
    while len(next_population) < population_cap:
        tournament_elites = get_tournament_elites(population, tournament_size, num_elites)
        left, right = random.sample(tournament_elites, 2)
        children = crossover(left.script.tree, right.script.tree)
        for child_tree in children:
            child_tree = mutate(child_tree)
            player = PlayerWrapper.from_script(exec_tree(child_tree))
            next_population.append(player)
    population = next_population
    for player in population:
        player.reset_battles()

'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'



2020-04-12 17:45:53,987 - dc956f42 - WARNING - Trying to login as dc956f42, showdown returned  dc956f42@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,005 - ad846d6d - WARNING - Trying to login as ad846d6d, showdown returned  ad846d6d@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,006 - 6c31ad26 - WARNING - Trying to login as 6c31ad26, showdown returned  6c31ad26@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,008 - ae7063fe - WARNING - Trying to login as ae7063fe, showdown returned  ae7063fe@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,008 - 8589ef2f - WARNING - Trying to login as 8589ef2f, showdown returned  8589ef2f@! - this might prevent future action

2020-04-12 17:45:54,074 - 6ad8f9cd - WARNING - Trying to login as 6ad8f9cd, showdown returned  6ad8f9cd@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,075 - 91dd520f - WARNING - Trying to login as 91dd520f, showdown returned  91dd520f@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,076 - 53c19886 - WARNING - Trying to login as 53c19886, showdown returned  53c19886@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,077 - 5c4fd33f - WARNING - Trying to login as 5c4fd33f, showdown returned  5c4fd33f@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,078 - fb630fe2 - WARNING - Trying to login as fb630fe2, showdown returned  fb630fe2@! - this might prevent future action

2020-04-12 17:45:54,110 - ad7fd2eb - WARNING - Trying to login as ad7fd2eb, showdown returned  ad7fd2eb@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,110 - 55307646 - WARNING - Trying to login as 55307646, showdown returned  55307646@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,111 - d5a529ce - WARNING - Trying to login as d5a529ce, showdown returned  d5a529ce@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,112 - 86539472 - WARNING - Trying to login as 86539472, showdown returned  86539472@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,112 - d4779673 - WARNING - Trying to login as d4779673, showdown returned  d4779673@! - this might prevent future action

2020-04-12 17:45:54,157 - ea1a98f7 - WARNING - Trying to login as ea1a98f7, showdown returned  ea1a98f7@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,158 - 11adf64b - WARNING - Trying to login as 11adf64b, showdown returned  11adf64b@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,159 - 7d49c604 - WARNING - Trying to login as 7d49c604, showdown returned  7d49c604@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,160 - f3c38fa2 - WARNING - Trying to login as f3c38fa2, showdown returned  f3c38fa2@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,160 - fbaa08c1 - WARNING - Trying to login as fbaa08c1, showdown returned  fbaa08c1@! - this might prevent future action

...

2020-04-12 17:45:54,313 - 2b725f5a - WARNING - Trying to login as 2b725f5a, showdown returned  2b725f5a@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,314 - 379bac77 - WARNING - Trying to login as 379bac77, showdown returned  379bac77@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,314 - d4d81b66 - WARNING - Trying to login as d4d81b66, showdown returned  d4d81b66@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,314 - 48f76379 - WARNING - Trying to login as 48f76379, showdown returned  48f76379@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,315 - d76d0659 - WARNING - Trying to login as d76d0659, showdown returned  d76d0659@! - this might prevent future action

2020-04-12 17:45:54,335 - 1f94511d - WARNING - Trying to login as 1f94511d, showdown returned  1f94511d@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,335 - e7f63cf0 - WARNING - Trying to login as e7f63cf0, showdown returned  e7f63cf0@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,337 - 21ab860d - WARNING - Trying to login as 21ab860d, showdown returned  21ab860d@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,337 - b973a50b - WARNING - Trying to login as b973a50b, showdown returned  b973a50b@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,338 - a8bbce40 - WARNING - Trying to login as a8bbce40, showdown returned  a8bbce40@! - this might prevent future action

2020-04-12 17:45:54,356 - 6ef0dae2 - WARNING - Trying to login as 6ef0dae2, showdown returned  6ef0dae2@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,357 - d869c342 - WARNING - Trying to login as d869c342, showdown returned  d869c342@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,357 - a0c56783 - WARNING - Trying to login as a0c56783, showdown returned  a0c56783@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,358 - 8f58e82e - WARNING - Trying to login as 8f58e82e, showdown returned  8f58e82e@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,358 - eaca7dfb - WARNING - Trying to login as eaca7dfb, showdown returned  eaca7dfb@! - this might prevent future action

2020-04-12 17:45:54,431 - 2b075983 - WARNING - Trying to login as 2b075983, showdown returned  2b075983@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,432 - 4dc03b0b - WARNING - Trying to login as 4dc03b0b, showdown returned  4dc03b0b@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,433 - e0e93e17 - WARNING - Trying to login as e0e93e17, showdown returned  e0e93e17@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,434 - 35514bef - WARNING - Trying to login as 35514bef, showdown returned  35514bef@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,435 - 6410b620 - WARNING - Trying to login as 6410b620, showdown returned  6410b620@! - this might prevent future action

2020-04-12 17:45:54,454 - 775a5fa3 - WARNING - Trying to login as 775a5fa3, showdown returned  775a5fa3@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,455 - 7c82754e - WARNING - Trying to login as 7c82754e, showdown returned  7c82754e@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,456 - bfe3aadb - WARNING - Trying to login as bfe3aadb, showdown returned  bfe3aadb@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,457 - 362d0b0b - WARNING - Trying to login as 362d0b0b, showdown returned  362d0b0b@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-12 17:45:54,458 - 488fe8fa - WARNING - Trying to login as 488fe8fa, showdown returned  488fe8fa@! - this might prevent future action

# Evaluate

In [83]:
players = []
script_to_gen = {}
for i, gen in enumerate(generations):
    for script in gen:
        script_to_gen[script] = i

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<PlayerNetwork.listen() done, defined at /Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py:214> exception=gaierror(8, 'nodename nor servname provided, or not known')>
Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/poke_env/player/player_network_interface.py", line 219, in listen
    async with websockets.connect(self.websocket_url) as websocket:
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 517, in __aenter__
    return await self
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/websockets/client.py", line 535, in __await_impl__
    transport, protocol = await self._create_connection()
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/

In [79]:
await evaluate_population(players, num_games=20, verbose=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    last_expr = (await self._async_exec(code_obj, self.user_ns))
  File "<ipython-input-79-6245eb2eb0a2>", line 4, in async-def-wrapper
  File "<ipython-input-3-4739a7399340>", line 20, in evaluate_population
    await asyncio.gather(*awaitables)
concurrent.futures._base.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-BaGiij6x/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'CancelledError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Uduse/.local/share/virtualenvs/final-B

CancelledError: 

In [ ]:
assign_trueskill(players)

In [ ]:
gen_ratings = [[] for _ in range(len(generations))]
for player in players:
    gen_idx = player_to_gen[player]
    gen_ratings[gen_idx].append(player.rating.mu)

In [ ]:
xs = list(range(len(gen_ratings)))
ys = np.mean(np.asarray(gen_ratings), -1)

In [ ]:
from bokeh.plotting import figure, output_file, show, reset_output, output_notebook

reset_output()
output_notebook()

p = figure(plot_width=800, plot_height=400)
p.line(xs, ys, line_width=2)

show(p)

In [ ]:
best_player = sorted(players, key=lambda p: p.rating.mu, reverse=True)[0]
print(best_player.rating)
print(best_player.script.raw_script)

In [ ]:
unfinished = []
for p in population:
    for b in p.battles.values():
        if not b.finished:
            unfinished.append(b)

In [ ]:
lk = {p.username: p for p in population}